In [49]:
import os
import pandas as pd
import requests
from requests.adapters import HTTPAdapter, Retry

# 📁 Create output folder
output_dir = "insane_data"
os.makedirs(output_dir, exist_ok=True)

# 🌐 Configure a session with retries
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retries)
session.mount("https://", adapter)
session.mount("http://", adapter)

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.nba.com/",
    "Origin": "https://www.nba.com",
    "Accept": "application/json",
    "Connection": "keep-alive",
}

# -----------------------------
# Helper Functions (use session.get + timeout + raise_for_status)
# -----------------------------
def fetch_all_players_stats(season="2024-25", date_to=""):
    url = "https://stats.nba.com/stats/leaguedashplayerstats"
    params = {
        "MeasureType": "Base", "PerMode": "PerGame",
        "Season": season, "SeasonType": "Regular Season",
        "LastNGames": "0", "DateFrom": "", "DateTo": date_to,
        "LeagueID": "00", "Location": "", "GameScope": "",
        "GameSegment": "", "Height": "", "ISTRound": "",
        "Month": "0", "OpponentTeamID": "0", "Outcome": "",
        "PORound": "0", "PaceAdjust": "N", "Period": "0",
        "PlayerExperience": "", "PlayerPosition": "",
        "PlusMinus": "N", "Rank": "N", "ShotClockRange": "",
        "StarterBench": "", "TeamID": "0", "TwoWay": "0",
        "VsConference": "", "VsDivision": ""
    }
    resp = session.get(url, params=params, headers=headers, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    return pd.DataFrame(data["resultSets"][0]["rowSet"],
                        columns=data["resultSets"][0]["headers"])

def fetch_player_game_logs(player_id, season="2024-25"):
    url = "https://stats.nba.com/stats/playergamelog"
    params = {"PlayerID": player_id, "Season": season, "SeasonType": "Regular Season"}
    resp = session.get(url, params=params, headers=headers, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    return pd.DataFrame(data["resultSets"][0]["rowSet"],
                        columns=data["resultSets"][0]["headers"])

def fetch_team_stats(measure_type, team_id="0", date_to=""):
    url = "https://stats.nba.com/stats/leaguedashteamstats"
    params = {
        "Conference": "", "DateFrom": "", "DateTo": date_to, "Division": "",
        "GameScope": "", "GameSegment": "", "Height": "", "ISTRound": "",
        "LastNGames": "8", "LeagueID": "00", "Location": "",
        "MeasureType": measure_type, "Month": "0", "OpponentTeamID": "0",
        "Outcome": "", "PORound": "0", "PaceAdjust": "N", "PerMode": "PerGame",
        "Period": "0", "PlayerExperience": "", "PlayerPosition": "",
        "PlusMinus": "N", "Rank": "N", "ShotClockRange": "",
        "StarterBench": "", "TeamID": team_id, "TwoWay": "0",
        "VsConference": "", "VsDivision": ""
    }
    resp = session.get(url, params=params, headers=headers, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    return pd.DataFrame(data["resultSets"][0]["rowSet"],
                        columns=data["resultSets"][0]["headers"])

def fetch_player_agg_stats(measure_type, player_id, date_to=""):
    url = "https://stats.nba.com/stats/leaguedashplayerstats"
    params = {
        "PlayerID": player_id, "LastNGames": "8", "Season": "2024-25",
        "SeasonType": "Regular Season", "MeasureType": measure_type,
        "DateFrom": "", "DateTo": date_to, "Conference": "",
        "Division": "", "GameScope": "", "GameSegment": "",
        "Height": "", "ISTRound": "", "Month": "0",
        "OpponentTeamID": "0", "Outcome": "", "PORound": "0",
        "PaceAdjust": "N", "PerMode": "PerGame", "Period": "0",
        "PlayerExperience": "", "PlayerPosition": "", "PlusMinus": "N",
        "Rank": "N", "Location": "", "ShotClockRange": "",
        "StarterBench": "", "TwoWay": "0", "VsConference": "",
        "VsDivision": ""
    }
    resp = session.get(url, params=params, headers=headers, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    return pd.DataFrame(data["resultSets"][0]["rowSet"],
                        columns=data["resultSets"][0]["headers"])

# -----------------------------
# Team ID Mapping
# -----------------------------
team_abbrev_to_id = {
    "ATL": "1610612737", "BOS": "1610612738", "BKN": "1610612751", "CHA": "1610612766",
    "CHI": "1610612741", "CLE": "1610612739", "DAL": "1610612742", "DEN": "1610612743",
    "DET": "1610612765", "GSW": "1610612744", "HOU": "1610612745", "IND": "1610612754",
    "LAC": "1610612746", "LAL": "1610612747", "MEM": "1610612763", "MIA": "1610612748",
    "MIL": "1610612749", "MIN": "1610612750", "NOP": "1610612740", "NYK": "1610612752",
    "OKC": "1610612760", "ORL": "1610612753", "PHI": "1610612755", "PHX": "1610612756",
    "POR": "1610612757", "SAC": "1610612758", "SAS": "1610612759", "TOR": "1610612761",
    "UTA": "1610612762", "WAS": "1610612764"
}

# -----------------------------
# Main Logic
# -----------------------------
season = "2024-25"

print("📊 Fetching all NBA players' stats...")
all_stats = fetch_all_players_stats(season)
all_stats = all_stats[all_stats["GP"] > 5]
min_cutoff = all_stats["MIN"].median()
relevant_players = all_stats[all_stats["MIN"] >= min_cutoff]
print(f"✅ Found {len(relevant_players)} relevant players based on Minutes Played.")

for _, player in relevant_players.iterrows():
    player_id   = player["PLAYER_ID"]
    player_name = player["PLAYER_NAME"]
    # skip if already done
    out_file = os.path.join(output_dir, f"{player_name.replace(' ','_')}_{player_id}.csv")
    if os.path.exists(out_file):
        print(f"⚠️ Skipping {player_name}, output exists.")
        continue

    print(f"📊 Processing {player_name}...")
    player_rows = []
    try:
        logs = fetch_player_game_logs(player_id, season)
        logs["GAME_DATE"] = pd.to_datetime(logs["GAME_DATE"], errors="coerce")
        logs.sort_values("GAME_DATE", inplace=True)

        for _, game in logs.iterrows():
            if pd.isna(game["GAME_DATE"]):
                continue
            game_date = game["GAME_DATE"].strftime("%Y-%m-%d")
            matchup   = game["MATCHUP"]

            # parse opponent
            if "vs." in matchup:
                opp = matchup.split("vs.")[1].strip().split()[0]
            elif "@" in matchup:
                opp = matchup.split("@")[1].strip().split()[0]
            else:
                continue
            if opp not in team_abbrev_to_id:
                continue
            opp_id = team_abbrev_to_id[opp]

            # fetch opponent team stats
            base_opp = fetch_team_stats("Base", opp_id, game_date)
            adv_opp  = fetch_team_stats("Advanced", opp_id, game_date)
            opp_stats = {}
            if not base_opp.empty and not adv_opp.empty:
                merged = pd.merge(base_opp, adv_opp, on="TEAM_ID", suffixes=("_Base","_Adv"))
                opp_stats = {f"opp_{k}": v for k, v in merged.iloc[0].to_dict().items()}

            # fetch player aggregated stats
            base_p = fetch_player_agg_stats("Base", player_id, game_date)
            adv_p  = fetch_player_agg_stats("Advanced", player_id, game_date)
            player_stats = {}
            if not base_p.empty and not adv_p.empty:
                merged_p = pd.merge(base_p, adv_p, on="PLAYER_ID", suffixes=("_Base","_Adv"))
                player_stats = {f"player_{k}": v for k, v in merged_p.iloc[0].to_dict().items()}

            # combine
            row = {
                "PLAYER_NAME": player_name,
                "PLAYER_ID": player_id,
                "GAME_DATE": game_date,
                "GAME_ID": game["Game_ID"],
                "PTS": game["PTS"],
                "REB": game["REB"],
                "AST": game["AST"],
                **opp_stats,
                **player_stats
            }
            player_rows.append(row)

    except Exception as e:
        print(f"❌ Error processing {player_name}: {e}")
        continue

    # write per-player CSV
    if player_rows:
        pd.DataFrame(player_rows).to_csv(out_file, index=False)
        print(f"✅ Wrote {len(player_rows)} rows for {player_name} → {out_file}")

print("🎉 All done.")




🏀 Processing season 2024-25…
  • 267 players ≥ median minutes (20.1)
📊 Processing AJ Green...
✅ Wrote 73 rows for AJ Green → insane_data/2024-25/AJ_Green_1631260.csv
📊 Processing AJ Johnson...


KeyboardInterrupt: 

In [2]:
print(lebron_game_logs.columns.tolist())

NameError: name 'lebron_game_logs' is not defined

In [ ]:
print(lebron_game_logs["GAME_DATE"].head())

In [50]:
import os
import glob
import pandas as pd
import numpy as np

# 📁 Input & output directories
input_dir  = "insane_data"
output_dir = "cropped_data_with_avgs_slopes"
os.makedirs(output_dir, exist_ok=True)

# ✂️ Specify which columns to KEEP
columns_to_keep = [
    "PLAYER_NAME",
    "PLAYER_ID",
    "GAME_DATE",
    "GAME_ID",
    "PTS",
    "REB",
    "AST",
    # opponent Base box-score metrics
    "opp_TEAM_ID",
    "opp_TEAM_NAME_Base",
    # opponent Advanced efficiency & pace
    "opp_E_OFF_RATING",
    "opp_E_DEF_RATING",
    "opp_E_NET_RATING",
    "opp_REB_PCT",
]

# ── Helper for slope calculation ────────────────────────────────────────────
def calc_slope(x):
    # x is a 1D array of length 8
    if np.any(np.isnan(x)):
        return np.nan
    return np.polyfit(np.arange(len(x)), x, 1)[0]

# ── Loop through every CSV in insane_data/ ───────────────────────────────────
for filepath in glob.glob(os.path.join(input_dir, "*.csv")):
    df = pd.read_csv(filepath, parse_dates=["GAME_DATE"])
    
    # 🔍 Only retain columns that actually exist
    keep = [c for c in columns_to_keep if c in df.columns]
    cropped = df[keep].copy()

    # 📅 Ensure chronological order by game date
    cropped.sort_values('GAME_DATE', inplace=True)

    # 📈 Calculate 8-game running averages and slopes (previous 8 games), shifted by 1
    for stat in ["PTS", "REB", "AST"]:
        cropped[f"{stat}_8_game_avg"] = (
            cropped[stat]
            .rolling(window=8)
            .mean()
            .shift(1)
        )
        cropped[f"{stat}_8_game_slope"] = (
            cropped[stat]
            .rolling(window=8)
            .apply(calc_slope, raw=True)
            .shift(1)
        )

    # 📝 Save to new file
    filename = os.path.basename(filepath)
    out_path = os.path.join(output_dir, filename)
    cropped.to_csv(out_path, index=False)
    
    print(f"✅ Processed {filename}, added avg & slope cols → saved to {out_path}")


✅ Processed Tim_Hardaway_Jr._203501.csv, added avg & slope cols → saved to cropped_data_with_avgs_slopes/Tim_Hardaway_Jr._203501.csv
✅ Processed Darius_Garland_1629636.csv, added avg & slope cols → saved to cropped_data_with_avgs_slopes/Darius_Garland_1629636.csv
✅ Processed Aaron_Gordon_203932 (2).csv, added avg & slope cols → saved to cropped_data_with_avgs_slopes/Aaron_Gordon_203932 (2).csv
✅ Processed Donte_DiVincenzo_1628978.csv, added avg & slope cols → saved to cropped_data_with_avgs_slopes/Donte_DiVincenzo_1628978.csv
✅ Processed KJ_Martin_1630231.csv, added avg & slope cols → saved to cropped_data_with_avgs_slopes/KJ_Martin_1630231.csv
✅ Processed Coby_White_1629632.csv, added avg & slope cols → saved to cropped_data_with_avgs_slopes/Coby_White_1629632.csv
✅ Processed Jordan_Hawkins_1641722.csv, added avg & slope cols → saved to cropped_data_with_avgs_slopes/Jordan_Hawkins_1641722.csv
✅ Processed Buddy_Hield_1627741.csv, added avg & slope cols → saved to cropped_data_with_avgs

In [51]:
import os
import glob
import pandas as pd

# 📁 Directory containing processed files
input_dir = "cropped_data_with_avgs_slopes"

# 📝 Path for the combined CV file
output_file = "combined_cv.csv"

# ── Read & concat all CSVs ─────────────────────────────────────────────────
all_files = glob.glob(os.path.join(input_dir, "*.csv"))
dfs = []
for f in all_files:
    df = pd.read_csv(f, parse_dates=["GAME_DATE"])
    dfs.append(df)

combined = pd.concat(dfs, ignore_index=True)

# 📅 Sort by date for time-series CV
combined.sort_values("GAME_DATE", inplace=True)

# 💾 Save the combined dataset
combined.to_csv(output_file, index=False)

print(f"✅ Combined {len(all_files)} files into '{output_file}', sorted by GAME_DATE.")


✅ Combined 270 files into 'combined_cv.csv', sorted by GAME_DATE.


In [52]:
import os
import pandas as pd

# 📁 Input combined file
input_file = "combined_cv.csv"

# 📁 Output directory for CV folds
output_dir = "cv_folds"
os.makedirs(output_dir, exist_ok=True)

# ── Load the combined dataset ────────────────────────────────────────────────
df = pd.read_csv(input_file, parse_dates=["GAME_DATE"])

# ── Define the 11 calendar intervals ─────────────────────────────────────────
intervals = [
    ("2024-10-22", "2024-11-06"),  # Interval 1
    ("2024-11-07", "2024-11-22"),  # Interval 2
    ("2024-11-23", "2024-12-08"),  # Interval 3
    ("2024-12-09", "2024-12-24"),  # Interval 4
    ("2024-12-25", "2025-01-09"),  # Interval 5
    ("2025-01-10", "2025-01-25"),  # Interval 6
    ("2025-01-26", "2025-02-10"),  # Interval 7
    ("2025-02-11", "2025-02-26"),  # Interval 8
    ("2025-02-27", "2025-03-14"),  # Interval 9
    ("2025-03-15", "2025-03-30"),  # Interval 10
    ("2025-03-31", "2025-04-13"),  # Interval 11
]

# ── Create the 10 folds ─────────────────────────────────────────────────────
for fold in range(1, 11):
    # Training uses intervals[0] up to intervals[fold-1]
    train_intervals = intervals[:fold]
    # Testing uses interval at index fold
    test_interval = intervals[fold]
    
    # Build masks
    train_mask = pd.Series(False, index=df.index)
    for start, end in train_intervals:
        mask = (df["GAME_DATE"] >= pd.to_datetime(start)) & (df["GAME_DATE"] <= pd.to_datetime(end))
        train_mask |= mask
    
    test_mask = (df["GAME_DATE"] >= pd.to_datetime(test_interval[0])) & \
                (df["GAME_DATE"] <= pd.to_datetime(test_interval[1]))
    
    # Subset
    train_df = df[train_mask]
    test_df  = df[test_mask]
    
    # Save to CSV
    train_path = os.path.join(output_dir, f"fold_{fold}_train.csv")
    test_path  = os.path.join(output_dir, f"fold_{fold}_test.csv")
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path,  index=False)
    
    print(f"Fold {fold}: train {len(train_df)} rows → {train_path}; "
          f"test {len(test_df)} rows → {test_path}")


Fold 1: train 1670 rows → cv_folds/fold_1_train.csv; test 1524 rows → cv_folds/fold_1_test.csv
Fold 2: train 3194 rows → cv_folds/fold_2_train.csv; test 1607 rows → cv_folds/fold_2_test.csv
Fold 3: train 4801 rows → cv_folds/fold_3_train.csv; test 1055 rows → cv_folds/fold_3_test.csv
Fold 4: train 5856 rows → cv_folds/fold_4_train.csv; test 1600 rows → cv_folds/fold_4_test.csv
Fold 5: train 7456 rows → cv_folds/fold_5_train.csv; test 1568 rows → cv_folds/fold_5_test.csv
Fold 6: train 9024 rows → cv_folds/fold_6_train.csv; test 1538 rows → cv_folds/fold_6_test.csv
Fold 7: train 10562 rows → cv_folds/fold_7_train.csv; test 1101 rows → cv_folds/fold_7_test.csv
Fold 8: train 11663 rows → cv_folds/fold_8_train.csv; test 1560 rows → cv_folds/fold_8_test.csv
Fold 9: train 13223 rows → cv_folds/fold_9_train.csv; test 1531 rows → cv_folds/fold_9_test.csv
Fold 10: train 14754 rows → cv_folds/fold_10_train.csv; test 1284 rows → cv_folds/fold_10_test.csv
